# Домашня робота №5

# Математичне сподівання та дисперсія

## 1 Порівняння

1. Реалізуйте функцію для підрахунку математичного сподівання випадкової величини $\xi$, заданої в класі **scipy.stats**, усіма можливими способами:
   - за допомогою методу **mean**;
   - за допомогою методу **expect**;
   - за допомогою методу **stats**.

2. Реалізуйте функцію для підрахунку дисперсії випадкової величини $\xi$, заданої в класі **scipy.stats**, усіма можливими способами:
   - за допомогою методу **var**;
   - за допомогою методу **expect** та формули для дисперсії
$\mathsf{D}(\xi) = \mathsf{E} \left[ \left( \xi - \mathsf{E}[\xi] \right)^2 \right]$;
   - за допомогою методу **expect** та формули для дисперсії
$\mathsf{D}(\xi) = \mathsf{E}[\xi^2] - \left( \mathsf{E}[\xi] \right)^2$;
   - за допомогою методу **stats**.

3. Порахуйте математичне сподівання та дисперсію, використовуючи функції з пунктів **1., 2.**, і порівняйте (виведіть різницю) їх із теоретичними значеннями, використовуючи формули з лекцій, для таких випадкових величин(оберіть 2 на вибір):
   - $\xi \sim Binom(10, 0.4)$ (біноміальний розподіл);
   - $\xi \sim Geom(0.5)$ (геометричний розподіл);
   - $\xi \sim Pois(0.6)$ (розподіл Пуассона);
   - $\xi \sim NBinom(5, 0.4)$ (від'ємний біноміальний розподіл);
   - $\xi \sim HGeom(20, 7, 12)$ (гіпергеометричний розподіл);
   - $\xi \sim Unif(0, 1)$ (рівномірний розподіл);
   - $\xi \sim Exp(0.5)$ (експоненційний (показниковий) розподіл);
   - $\xi \sim N(0, 1)$ (нормальний розподіл);
   - $\xi \sim \Gamma(2)$ (гамма розподіл);
   - $\xi \sim LogNorm(0, 1)$ (логнормальний розподіл);
   - $\xi \sim B(2, 1.1)$ (бета розподіл).

In [8]:
from scipy.stats import binom, expon
import numpy as np
import pandas as pd

def compute_expectations(rv):
    mean_method = rv.mean()
    mean_expect = rv.expect()
    mean_stats = rv.stats(moments='m')
    return mean_method, mean_expect, mean_stats

def compute_variances(rv):
    var_method = rv.var()
    mean = rv.expect()
    var_expect_formula1 = rv.expect(lambda x: (x - mean) ** 2)
    var_expect_formula2 = rv.expect(lambda x: x**2) - mean ** 2
    var_stats = rv.stats(moments='v')
    return var_method, var_expect_formula1, var_expect_formula2, var_stats

# Вибрані розподіли: Binom(10, 0.4) і Exp(0.5)
distributions = {
    "Binom(10, 0.4)": binom(n=10, p=0.4),
    "Exp(0.5)": expon(scale=1/0.5)
}

results = []

for name, dist in distributions.items():
    mean_vals = compute_expectations(dist)
    var_vals = compute_variances(dist)

    theo_mean = mean_vals[0]
    theo_var = var_vals[0]

    results.append({
        'Розподіл': name,
        'mean()': mean_vals[0],
        'expect()': mean_vals[1],
        'stats(m)': mean_vals[2],
        'Δ_μ_expect': abs(mean_vals[1] - theo_mean),
        'Δ_μ_stats': abs(mean_vals[2] - theo_mean),
        'var()': var_vals[0],
        'expect формула 1': var_vals[1],
        'expect формула 2': var_vals[2],
        'stats(v)': var_vals[3],
        'Δ_σ²_expect1': abs(var_vals[1] - theo_var),
        'Δ_σ²_expect2': abs(var_vals[2] - theo_var),
        'Δ_σ²_stats': abs(var_vals[3] - theo_var),
    })

df = pd.DataFrame(results)
print(df.to_string(index=False))


      Розподіл  mean()  expect()  stats(m)   Δ_μ_expect  Δ_μ_stats  var()  expect формула 1  expect формула 2  stats(v)  Δ_σ²_expect1  Δ_σ²_expect2  Δ_σ²_stats
Binom(10, 0.4)     4.0       4.0       4.0 1.776357e-15        0.0    2.4               2.4               2.4       2.4  1.776357e-15  9.769963e-15         0.0
      Exp(0.5)     2.0       2.0       2.0 0.000000e+00        0.0    4.0               4.0               4.0       4.0  2.664535e-15  3.552714e-15         0.0


## 2 Унікальні цифри

Нехай випадковим чином вибирають послідовно $n$ цифр (цифри можуть повторюватися, вибірка з поверненням).
Нехай тоді випадкова величина $\xi$ - це кількість унікальних цифр в отриманому наборі.

Напишіть функцію для обрахунку математичного сподівання та дисперсії випадкової величини $\xi$ в залежності від кількості $n$ цифр у наборі методом повного перебору.
Виведіть результат для $n = 1, 2, 3, 4, 5, 6, 7.$

In [6]:
import itertools
import numpy as np
import pandas as pd
from collections import Counter

def unique_digit_stats_full(n):
    digits = list(range(10))
    all_sequences = itertools.product(digits, repeat=n)
    count_dict = Counter()

    for seq in all_sequences:
        unique_count = len(set(seq))
        count_dict[unique_count] += 1

    total = 10**n
    values = np.array(list(count_dict.keys()))
    counts = np.array(list(count_dict.values()))
    probs = counts / total

    E_xi = np.sum(values * probs)
    D_xi = np.sum((values - E_xi) ** 2 * probs)

    return E_xi, D_xi

results_full = []
for n in range(1, 8):
    E, D = unique_digit_stats_full(n)
    results_full.append({'n': n, 'E[ξ]': E, 'D[ξ]': D})

df_full = pd.DataFrame(results_full)
print(df_full.to_string(index=False))


 n     E[ξ]     D[ξ]
 1 1.000000 0.000000
 2 1.900000 0.090000
 3 2.710000 0.225900
 4 3.439000 0.378279
 5 4.095100 0.528256
 6 4.685590 0.664416
 7 5.217031 0.780545


## 3 Парадокс днів народження

Нехай маємо $n$ людей.
Їх дні народження припадають на один з 12 місяців.
Нехай тоді випадкова величина $\xi$ - це кількість співпадінь місяців народжень серед цих $n$ людей.

Напишіть функцію для обрахунку математичного сподівання та дисперсії випадкової величини $\xi$ в залежності від кількості $n$ людей методом повного перебору.
Виведіть результат для $n = 1, 2, 3, 4, 5, 6.$

Для якого $n$ виконується $\mathsf{E}\left[ \xi \right] \geq 1$?

**Зауваження.** Кількість співпадінь означає кількість однакових пар. Наприклад,
- у наборі $\{ 1, 1, 2, 3 \}$ є одна пара;
- у наборі $\{ 1, 1, 1, 3 \}$ є три пари;
- у наборі $\{ 1, 1, 1, 1 \}$ є шість пар.

In [7]:
import itertools
from collections import Counter
import pandas as pd
import numpy as np

def birthday_stats(n, months=12):
    all_birthdays = itertools.product(range(1, months + 1), repeat=n)
    total_sequences = 0
    total_pairs = []

    for bdays in all_birthdays:
        counter = Counter(bdays)
        pairs = sum(v * (v - 1) // 2 for v in counter.values())
        total_pairs.append(pairs)
        total_sequences += 1

    total_pairs = np.array(total_pairs)
    E_xi = np.mean(total_pairs)
    D_xi = np.var(total_pairs)

    return E_xi, D_xi

results = []
for n in range(1, 7):
    E, D = birthday_stats(n)
    results.append({'n': n, 'E[ξ]': E, 'D[ξ]': D})

df_birthday = pd.DataFrame(results)
print(df_birthday.to_string(index=False))

n_ge_1 = next((row['n'] for row in results if row['E[ξ]'] >= 1), None)
print(f"\nНайменше n, де E[ξ] ≥ 1: n = {n_ge_1}")


 n     E[ξ]     D[ξ]
 1 0.000000 0.000000
 2 0.083333 0.076389
 3 0.250000 0.229167
 4 0.500000 0.458333
 5 0.833333 0.763889
 6 1.250000 1.145833

Найменше n, де E[ξ] ≥ 1: n = 6
